In [188]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import warnings
import csv
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", None)

In [127]:
#Get html
url = "https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats"
html = requests.get(url=url).text

soup = BeautifulSoup(html, "html.parser")
table = soup.find("tbody")
rows = table.find_all("tr")

cols = ['player_id', 'name', 'country', 'position', 'club', 'league', 'games_played',\
       'games_started', 'minutes_played', 'minutes_per_90', 'goals', 'assists',\
       'non_penalty_goals', 'penalty_goals', 'penalty_attempted', 'yellow_cards',\
       'red_card', 'goals_per_90', 'assits_per_90', 'non_penalty_goals_per_90']


KeyboardInterrupt: 

In [283]:
#create a dataframe with player stats
df = pd.DataFrame(columns=cols)

for row in rows:
    try:
        if row['class']:
            continue
    except:
        player_id = int(row.find("th").text)
        values = row.find_all("td")
        name = values[0].find('a').text
        country = values[1].text.split(' ')[len(values[1].text.split(' ')) - 1]
        position = values[2].text.split(',')
        club = values[3].text
        league = values[4].find('a').text
        games_played = float(values[7].text)
        games_started = float(values[8].text)
        minutes_played = float(''.join(values[9].text.split(',')))
        minutes_per_90 = float(values[10].text)
        goals = float(values[11].text) 
        assists = float(values[12].text)
        non_penalty_goals = float(values[13].text)
        penalty_goals = float(values[14].text)
        penalty_attempted = float(values[15].text)
        yellow_cards = float(values[16].text)
        red_card = float(values[17].text)
        goals_per_90 = float(values[18].text)
        assits_per_90 = float(values[19].text)
        non_penalty_goals_per_90 = float(values[20].text)

        # stats.append([index, name, country, position, club, league, games_played, games_started, minutes_played, minutes_per_90, goals, assists, non_penalty_goals, penalty_goals, penalty_attempted, yellow_cards, red_card, goals_per_90, assits_per_90, non_penalty_goals_per_90])
        
        stats = [player_id, name, country, position, club, league, games_played, games_started,\
                                minutes_played, minutes_per_90, goals, assists, non_penalty_goals, penalty_goals,\
                                penalty_attempted, yellow_cards, red_card, goals_per_90, assits_per_90,\
                                non_penalty_goals_per_90]
#         pd.DataFrame([stats], columns=cols).head()
#         df = df.append(pd.DataFrame([stats], columns=cols))
#         df = pd.conca([df, pd.DataFrame([index, name, country, position, club, league, games_played, games_started, minutes_played, minutes_per_90, goals, assists, non_penalty_goals, penalty_goals, penalty_attempted, yellow_cards, red_card, goals_per_90, assits_per_90, non_penalty_goals_per_90])], axis=0, ignore_index = True)
        df = pd.concat([df, pd.DataFrame([stats], columns=cols)])
    

player_list = df[['player_id', 'name', 'country','league', 'club', 'position']]    
player_list = player_list.set_index('player_id')
df = df.set_index('player_id')
df.head()

# df.to_csv("data.csv")


,name,country,position,club,league,games_played,games_started,minutes_played,minutes_per_90,goals,assists,non_penalty_goals,penalty_goals,penalty_attempted,yellow_cards,red_card,goals_per_90,assits_per_90,non_penalty_goals_per_90
player_id,,,,,,,,,,,,,,,,,,,
1,Brenden Aaronson,USA,"[MF, FW]",Leeds United,Premier League,14.0,14.0,1189.0,13.2,1.0,2.0,1.0,0.0,0.0,2.0,0.0,0.08,0.15,0.23
2,Yunis Abdelhamid,MAR,[DF],Reims,Ligue 1,15.0,15.0,1350.0,15.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00,0.00,0.00
3,Himad Abdelli,FRA,"[MF, FW]",Angers,Ligue 1,7.0,2.0,231.0,2.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
4,Salis Abdul Samed,GHA,[MF],Lens,Ligue 1,15.0,15.0,1349.0,15.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.07,0.00,0.07
5,Laurent Abergel,FRA,[MF],Lorient,Ligue 1,10.0,10.0,807.0,9.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.00,0.11,0.11


In [284]:

df = df.drop(['name', 'league', 'club', 'name', 'country'], axis = 1)

#one hot encoding position
mlb = MultiLabelBinarizer(sparse_output = True)
df = df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('position')),
                index=df.index,
                columns=mlb.classes_))

df.head()

,games_played,games_started,minutes_played,minutes_per_90,goals,assists,non_penalty_goals,penalty_goals,penalty_attempted,yellow_cards,red_card,goals_per_90,assits_per_90,non_penalty_goals_per_90,DF,FW,GK,MF
player_id,,,,,,,,,,,,,,,,,,
1,14.0,14.0,1189.0,13.2,1.0,2.0,1.0,0.0,0.0,2.0,0.0,0.08,0.15,0.23,0,1,0,1
2,15.0,15.0,1350.0,15.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00,0.00,0.00,1,0,0,0
3,7.0,2.0,231.0,2.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0,1,0,1
4,15.0,15.0,1349.0,15.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.07,0.00,0.07,0,0,0,1
5,10.0,10.0,807.0,9.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.00,0.11,0.11,0,0,0,1


In [285]:
player_list.head()

,name,country,league,club,position
player_id,,,,,
1,Brenden Aaronson,USA,Premier League,Leeds United,"[MF, FW]"
2,Yunis Abdelhamid,MAR,Ligue 1,Reims,[DF]
3,Himad Abdelli,FRA,Ligue 1,Angers,"[MF, FW]"
4,Salis Abdul Samed,GHA,Ligue 1,Lens,[MF]
5,Laurent Abergel,FRA,Ligue 1,Lorient,[MF]


In [331]:
def recommend_player(player_id, df):
    # player = df.query(f'index == {player_id}')
    temp = df.copy().drop(['GK', 'DF', "FW", "MF"], axis = 1)
#     global player_list
    
    player = df.iloc[player_id-1]
    player.head()
    player_pos = player[['GK', 'DF', 'MF', 'FW']]

    df['pos_similarity'] = df[['GK', 'DF', 'MF', 'FW']].dot(player_pos)
    df = df[df['pos_similarity'] > 0]

    # for z normalization of stats
    cols_for_norm = ['games_played', 'games_started',
           'minutes_played', 'minutes_per_90', 'goals', 'assists',
           'non_penalty_goals', 'penalty_goals', 'penalty_attempted',
           'yellow_cards', 'red_card', 'goals_per_90', 'assits_per_90',
           'non_penalty_goals_per_90']

    for col in cols_for_norm:
        df[col] = (df[col] - df[col].mean()) / df[col].std()

    player = df.loc[player_id]
    
    df.drop(['GK', 'DF', "FW", "MF"], axis = 1)
    
    #karl pearsons correlation
    pearsons = df.T.corr(method='pearson')
    
    #cosine similarity 
    cosine = cosine_similarity(df)
    cosine = pd.DataFrame(data=cosine, index = pearsons.columns, columns = pearsons.columns)
    
    final = (cosine + pearsons) / 2
    final.head()
    
    
    similar = final[player_id].sort_values(ascending = False)
    index = similar.index - 1

    return player_list.merge(temp, on='player_id').drop('pos_similarity', axis=1).iloc[index]

In [216]:
df.head()

,games_played,games_started,minutes_played,minutes_per_90,goals,assists,non_penalty_goals,penalty_goals,penalty_attempted,yellow_cards,red_card,goals_per_90,assits_per_90,non_penalty_goals_per_90,DF,FW,GK,MF,pos_similarity
player_id,,,,,,,,,,,,,,,,,,,
1,1.024980,1.655552,1.661730,1.659021,-0.095445,1.034019,-0.040218,-0.256006,-0.286923,0.525920,-0.249217,-0.245081,0.120322,-0.128076,0,1,0,1,2.0
3,-0.555160,-0.823787,-0.784714,-0.777077,-0.619150,-0.615940,-0.622193,-0.256006,-0.286923,-0.903653,-0.249217,-0.453202,-0.432132,-0.606132,0,1,0,1,2.0
4,1.250715,1.862164,2.070322,2.072698,-0.095445,-0.615940,-0.040218,-0.256006,-0.286923,0.525920,-0.249217,-0.271096,-0.432132,-0.460637,0,0,0,1,1.0
5,0.122043,0.829106,0.686217,0.693775,-0.619150,0.209040,-0.622193,-0.256006,-0.286923,0.525920,-0.249217,-0.453202,-0.026999,-0.377497,0,0,0,1,1.0
6,0.122043,-1.237010,-1.106480,-1.098825,-0.095445,-0.615940,-0.040218,-0.256006,-0.286923,-0.903653,-0.249217,1.784096,-0.432132,1.181382,0,1,0,1,2.0


In [250]:
#karl pearsons correlation cofficient
pearsons = df.T.corr(method='pearson')
pearsons.head()

player_id,1,3,4,5,6,7,8,10,12,13,...,2368,2369,2372,2373,2375,2379,2380,2381,2382,2383
player_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.275315,0.810753,0.866165,-0.113735,0.878315,0.617284,-0.105958,0.477780,0.373519,...,0.158581,0.389780,-0.195033,0.180764,0.652504,0.814701,-0.231922,0.275293,0.222042,0.670676
3,0.275315,1.000000,-0.015862,0.339453,0.696996,0.129679,0.043570,0.852772,-0.012966,0.009474,...,0.893187,-0.087704,0.533291,0.989424,-0.117311,-0.116410,0.085071,-0.233350,0.726140,0.190817
4,0.810753,-0.015862,1.000000,0.763241,-0.310518,0.624628,0.605851,-0.424850,0.492476,0.535528,...,-0.145315,0.058420,-0.587293,-0.123388,0.661979,0.781631,-0.406296,0.322092,-0.001646,0.715577
5,0.866165,0.339453,0.763241,1.000000,-0.129885,0.608768,0.214293,-0.030411,0.064807,0.511427,...,0.226092,0.159134,-0.214706,0.269424,0.376661,0.834518,-0.133388,0.016187,0.269926,0.662520
6,-0.113735,0.696996,-0.310518,-0.129885,1.000000,-0.084255,-0.016714,0.643484,0.006968,-0.299048,...,0.574679,-0.049512,0.713512,0.707452,-0.191838,-0.409126,0.191553,-0.079764,0.480886,-0.042973


In [252]:
#cosine similarity 
cosine = cosine_similarity(df)
cosine = pd.DataFrame(data=cosine, index = sim.columns, columns = sim.columns)
cosine.head()

player_id,1,3,4,5,6,7,8,10,12,13,...,2368,2369,2372,2373,2375,2379,2380,2381,2382,2383
player_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.057655,0.835802,0.815173,-0.067437,0.913526,0.733292,-0.391018,0.644293,0.505931,...,-0.164531,0.597213,-0.137513,-0.046589,0.781053,0.873829,-0.090729,0.555432,-0.011711,0.771674
3,0.057655,1.000000,-0.121951,0.259375,0.657732,-0.110890,-0.109357,0.835207,-0.161651,-0.098313,...,0.884925,-0.229274,0.502358,0.989098,-0.264727,-0.245127,0.035272,-0.350360,0.749194,0.008461
4,0.835802,-0.121951,1.000000,0.770198,-0.267903,0.681998,0.680993,-0.543044,0.594957,0.606822,...,-0.306547,0.275116,-0.524752,-0.234262,0.717786,0.816296,-0.302921,0.484203,-0.126488,0.766454
5,0.815173,0.259375,0.770198,1.000000,-0.117212,0.568194,0.292152,-0.144255,0.172388,0.544237,...,0.089386,0.253050,-0.201406,0.179241,0.419725,0.797983,-0.092496,0.157582,0.181136,0.662322
6,-0.067437,0.657732,-0.267903,-0.117212,1.000000,-0.026876,0.008951,0.512155,0.029935,-0.258135,...,0.482424,-0.014319,0.713963,0.655051,-0.116117,-0.315786,0.195967,-0.029483,0.441724,-0.012884


In [255]:
final = (cosine + pearsons) / 2
final.head()

player_id,1,3,4,5,6,7,8,10,12,13,...,2368,2369,2372,2373,2375,2379,2380,2381,2382,2383
player_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.166485,0.823278,0.840669,-0.090586,0.895921,0.675288,-0.248488,0.561036,0.439725,...,-0.002975,0.493497,-0.166273,0.067088,0.716779,0.844265,-0.161325,0.415363,0.105166,0.721175
3,0.166485,1.000000,-0.068907,0.299414,0.677364,0.009395,-0.032893,0.843989,-0.087309,-0.044420,...,0.889056,-0.158489,0.517824,0.989261,-0.191019,-0.180769,0.060171,-0.291855,0.737667,0.099639
4,0.823278,-0.068907,1.000000,0.766720,-0.289210,0.653313,0.643422,-0.483947,0.543717,0.571175,...,-0.225931,0.166768,-0.556022,-0.178825,0.689882,0.798964,-0.354609,0.403148,-0.064067,0.741015
5,0.840669,0.299414,0.766720,1.000000,-0.123549,0.588481,0.253222,-0.087333,0.118598,0.527832,...,0.157739,0.206092,-0.208056,0.224333,0.398193,0.816250,-0.112942,0.086884,0.225531,0.662421
6,-0.090586,0.677364,-0.289210,-0.123549,1.000000,-0.055566,-0.003882,0.577820,0.018451,-0.278591,...,0.528552,-0.031915,0.713737,0.681251,-0.153978,-0.362456,0.193760,-0.054623,0.461305,-0.027928


In [277]:
player_id = 1
similar = final[player_id].drop(player_id).sort_values(ascending = False)
index = similar.index - 1
similar.head()

player_id
874     0.973042
645     0.965911
1544    0.962674
712     0.938351
852     0.935145
Name: 1, dtype: float64

In [278]:
player_list.iloc[index]

,name,country,league,club,position
player_id,,,,,
874,Moi Gómez,ESP,La Liga,Osasuna,"[MF, FW]"
645,Ritsu Doan,JPN,Bundesliga,Freiburg,"[FW, MF]"
1544,Mason Mount,ENG,Premier League,Chelsea,"[MF, FW]"
712,Mohamed Elyounoussi,NOR,Premier League,Southampton,"[MF, FW]"
852,Morgan Gibbs-White,ENG,Premier League,Nott'ham Forest,"[FW, MF]"
385,Gianluca Caprari,ITA,Serie A,Monza,"[FW, MF]"
924,Mattéo Guendouzi,FRA,Ligue 1,Marseille,[MF]
1696,Mario Pašalić,CRO,Serie A,Atalanta,"[MF, FW]"
1016,Franck Honorat,FRA,Ligue 1,Brest,"[FW, MF]"


In [280]:
player_id = 1
similar = cosine[player_id].drop(player_id).sort_values(ascending = False)
index = similar.index - 1
similar.head()
player_list.iloc[index]

,name,country,league,club,position
player_id,,,,,
874,Moi Gómez,ESP,La Liga,Osasuna,"[MF, FW]"
645,Ritsu Doan,JPN,Bundesliga,Freiburg,"[FW, MF]"
1544,Mason Mount,ENG,Premier League,Chelsea,"[MF, FW]"
852,Morgan Gibbs-White,ENG,Premier League,Nott'ham Forest,"[FW, MF]"
924,Mattéo Guendouzi,FRA,Ligue 1,Marseille,[MF]
385,Gianluca Caprari,ITA,Serie A,Monza,"[FW, MF]"
1696,Mario Pašalić,CRO,Serie A,Atalanta,"[MF, FW]"
1016,Franck Honorat,FRA,Ligue 1,Brest,"[FW, MF]"
499,David Pereira da Costa,POR,Ligue 1,Lens,"[MF, FW]"


In [281]:
player_id = 1
similar = pearsons[player_id].drop(player_id).sort_values(ascending = False)
index = similar.index - 1
similar.head()
player_list.iloc[index]

,name,country,league,club,position
player_id,,,,,
874,Moi Gómez,ESP,La Liga,Osasuna,"[MF, FW]"
645,Ritsu Doan,JPN,Bundesliga,Freiburg,"[FW, MF]"
1544,Mason Mount,ENG,Premier League,Chelsea,"[MF, FW]"
712,Mohamed Elyounoussi,NOR,Premier League,Southampton,"[MF, FW]"
385,Gianluca Caprari,ITA,Serie A,Monza,"[FW, MF]"
852,Morgan Gibbs-White,ENG,Premier League,Nott'ham Forest,"[FW, MF]"
1728,Mathias Pereira Lage,POR,Ligue 1,Brest,"[MF, FW]"
924,Mattéo Guendouzi,FRA,Ligue 1,Marseille,[MF]
1696,Mario Pašalić,CRO,Serie A,Atalanta,"[MF, FW]"


In [333]:
p_id = 100
recommend_player(p_id, df)

,name,country,league,club,position,games_played,games_started,minutes_played,minutes_per_90,goals,assists,non_penalty_goals,penalty_goals,penalty_attempted,yellow_cards,red_card,goals_per_90,assits_per_90,non_penalty_goals_per_90
player_id,,,,,,,,,,,,,,,,,,,
100,Houssem Aouar,FRA,Ligue 1,Lyon,[MF],6.0,6.0,343.0,3.8,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.26,0.00,0.26
1967,Baptiste Santamaria,FRA,Ligue 1,Rennes,[MF],7.0,7.0,551.0,6.1,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.16,0.00,0.16
1245,Roméo Lavia,BEL,Premier League,Southampton,[MF],7.0,6.0,557.0,6.2,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.16,0.00,0.16
607,Abdou Diallo,SEN,Bundesliga,RB Leipzig,"[DF, MF]",5.0,5.0,436.0,4.8,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.21,0.00,0.21
1044,Asier Illarramendi,ESP,La Liga,Real Sociedad,[MF],6.0,2.0,153.0,1.7,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.59,0.00,0.59
1520,Florent Mollet,FRA,Bundesliga,Schalke 04,[MF],9.0,4.0,410.0,4.6,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.22,0.00,0.22
1224,Konrad Laimer,AUT,Bundesliga,RB Leipzig,[MF],4.0,4.0,334.0,3.7,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.00,0.00,0.00
283,Quentin Boisgard,FRA,Ligue 1,Lorient,[MF],5.0,1.0,184.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.49,0.00,0.49
99,Anuar,MAR,La Liga,Valladolid,"[MF, FW]",3.0,3.0,210.0,2.3,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.43,0.00,0.43
